# Transfer learning with transformers using Roberta large
by Artyom Glazunov

In this notebook you can find one example on how to use transfer learning with the transfermers library. There is also some information on how to load such models in kaggle notebooks, because the inference in some competition does not support Internet connection (the solution is to load some notebooks output with saved models in the inference notebook as an input). The notebook with saved Roberta model you can find here https://www.kaggle.com/artemglazunov1990/roberta-save, this model is used here to be finetuned on our regression task. The inference example you can find here https://www.kaggle.com/artemglazunov1990/inference-with-roberta

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Download pretrained Roberta large model with tokenizer

In [ ]:
%%bash
cp ../input/roberta-save/rob.zip .
cp ../input/roberta-save/rob_tok.zip .
unzip rob.zip
unzip rob_tok.zip 
rm -r rob.zip rob_tok.zip 

Import usefull packages

In [ ]:
import numpy as np
import pandas as pd
from transformers import RobertaTokenizer, RobertaModel, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split
import tqdm

Load the data

In [ ]:
data = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
data.head()

Get train and validation sets

In [ ]:
data_train, data_val, y_err_train, y_err_val = train_test_split(data['excerpt'].values, data[['target', 'standard_error']].values,
                                                        test_size=0.15,
                                                        random_state=42)
data_train.shape, data_val.shape

Create Roberta tokenizer

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(
    'rob_tok'
)

Encode data

In [ ]:
%%time
encoded_data_train = tokenizer.batch_encode_plus(
    data_train,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=512,
    return_tensors='pt',
)

encoded_data_val = tokenizer.batch_encode_plus(
    data_val,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=512,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
values_train = torch.tensor(y_err_train[:, 0],dtype=torch.float)
errors_train = torch.tensor(y_err_train[:, 1],dtype=torch.float)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
values_val = torch.tensor(y_err_val[:, 0], dtype=torch.float)
errors_val = torch.tensor(y_err_val[:, 1],dtype=torch.float)

As a result, we have pytorch tensors of padded ids lists (ids of tokens in our texts from our pretrained Roberta vocab), attention masks (to show the model where is our padding, we do not want it to shange the model's behavior), target and errors (here, it isn't used, but you can try to use it as an uncertainty level in your criterion later).

Let's create tensor datasets and, after that, dataloaders (iterators that will be providing us with batches)

In [ ]:
dataset_train = TensorDataset(input_ids_train,
                             attention_masks_train,
                             values_train,
                             errors_train)
dataset_val = TensorDataset(input_ids_val,
                            attention_masks_val,
                            values_val,
                            errors_val)

In [ ]:
batch_size = 4

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=2*batch_size
)

Create device (we will load our model and batches on it, but be carefull and watch the memory)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Let's initialize our model class, criterion and rmse function

In [ ]:
class BERTRegressor(torch.nn.Module): 
    def __init__(self, pretrained_src = 'rob'): 
        super().__init__()
        self.bert = RobertaModel.from_pretrained(pretrained_src)
        self.linear = torch.nn.Linear(1024, 1)
        self.dropout = torch.nn.Dropout(0.15)
        
    def forward(self, input_ids, attention_mask): #x - tokenized batch
        hidden = self.bert(input_ids, 
                           attention_mask=attention_mask)[0][:, 0, :]#CLS token output                                                          
        output = self.linear(self.dropout(hidden))
        return output


class RMSELoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = torch.nn.MSELoss()
        
    def forward(self,yhat,y):
        loss = torch.sqrt(self.mse(yhat,y))
        return loss

def rmse_metric(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))

Our params for training

In [ ]:
warm_prop = 0.1 # we want our learning rate to grow for a while
epochs = 8
clip = 1 #we do not want too big gradients

model = BERTRegressor().to(device)
criterion = RMSELoss()
optimizer = AdamW(
    model.parameters(),
    lr= 3e-5,#the original paper:2e-5 -> 5e-5
    eps=1e-8
)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(len(dataloader_train)*epochs * warm_prop),
    num_training_steps=len(dataloader_train)*epochs
)

Let's initialize our function for evaluation

In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm.notebook.tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]
        }
        target = batch[2]

        with torch.no_grad():        
            output = model(**inputs)
            
        loss = criterion(output, target.view(-1,1))
        loss_val_total += loss

        output = output.detach().cpu().numpy()
        target = target.cpu().numpy()
        predictions.append(output)
        true_vals.append(target)
    
    loss_val_avg = loss_val_total / len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

Let's start our transfer learning process

In [ ]:
best_val_loss = float('inf')
for epoch in tqdm.notebook.tqdm(range(epochs)):
    model.train()

    epoch_loss = 0
    for batch in tqdm.notebook.tqdm(dataloader_train):

        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1]
          }
        target = batch[2]

        optimizer.zero_grad()        

        output = model(**inputs)     
        loss = criterion(output, target.view(-1,1))      
        loss.backward()
        epoch_loss += loss.item()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)      
        optimizer.step()
        scheduler.step()     

    val_loss, predictions, true_vals = evaluate(dataloader_val)
    if val_loss < best_val_loss:        
        #here can be you code, if you want to save your best model
        pass
    train_loss = epoch_loss / len(dataloader_train)
    rmse_val = rmse_metric(true_vals, predictions)
    print('-------')
    print(f'Training loss: {train_loss}')
    print(f'Validation loss: {val_loss}')
    print(f"RMSE on validation: {rmse_val}")

In [ ]:
!rm -r rob rob_tok